In [1]:
from ngsolve import *
from ngsolve.meshes import Make1DMesh
from ngsolve.webgui import Draw
from scipy import random
import math
import scipy.linalg

importing NGSolve-6.2.2204


In [16]:
axial_load = 4

In [3]:
mesh = Make1DMesh(10, mapping=None, periodic=False)

In [4]:
psi = H1(mesh, order=1, dirichlet="left|right")
sigma = H1(mesh, order=1, dirichlet=" ")
mu = NumberSpace(mesh)
fesm = psi*sigma*mu
print ("ndof =", fesm.ndof)

ndof = 23


In [5]:
psi, sigma,mu = fesm.TrialFunction()
v , tau,lamda = fesm.TestFunction()

In [6]:
am = BilinearForm(fesm)
am += (grad(sigma)*grad(v) - axial_load*grad(psi)*grad(v) + \
       1/2*grad(psi)*grad(v)*mu + mu*lamda - grad(psi)**2*lamda + grad(psi)*grad(tau) \
       + sigma*tau)*dx

pre = Preconditioner(am, "multigrid")

m = BilinearForm(fesm)
m += psi*v*dx

am.Assemble()
m.Assemble()

psi = GridFunction(fesm)

In [7]:
r = psi.vec.CreateVector()
w = psi.vec.CreateVector()
Mu = psi.vec.CreateVector()
Au = psi.vec.CreateVector()
Mw = psi.vec.CreateVector()
Aw = psi.vec.CreateVector()

In [8]:
r.FV().NumPy()[:] = random.rand(fesm.ndof)
psi.vec.data = Projector(fesm.FreeDofs(), True) * r

In [15]:
for i in range(20):
    Au.data = am.mat * psi.vec
    Mu.data = m.mat * psi.vec
    auu = InnerProduct(Au, psi.vec)
    muu = InnerProduct(Mu, psi.vec)
    # Rayleigh quotient
    lam = auu/muu
   # print (lam / (math.pi**2))
    r.data = Au - lam * Mu
    w.data = pre.mat * r.data
    w.data = 1/Norm(w) * w
    Aw.data = am.mat * w
    Mw.data = m.mat * w
    asmall = Matrix(2,2)
    asmall[0,0] = auu
    asmall[0,1] = asmall[1,0] = InnerProduct(Au, w)
    asmall[1,1] = InnerProduct(Aw, w)
    msmall = Matrix(2,2)
    msmall[0,0] = muu
    msmall[0,1] = msmall[1,0] = InnerProduct(Mu, w)
    msmall[1,1] = InnerProduct(Mw, w)
    # print ("asmall =", asmall, ", msmall = ", msmall)
    
    eval,evec = scipy.linalg.eigh(a=asmall, b=msmall)
    # print (eval, evec)
    psi.vec.data = float(evec[0,0]) * psi.vec + float(evec[1,0]) * w
    

print(psi.vec.data)

      -0
 0.0635751
 0.504372
 0.132925
 1.25286
 0.385571
 -2.18924
 -0.139783
 -0.429656
 2.91541
      -0
 -19.7733
 -13.3243
 -14.6651
 -13.869
 -34.0913
 40.0796
 135.976
 60.4478
 -48.7956
 -127.803
 80.4974
 2.55509


